In [1]:
import sys
sys.path.insert(0,'/home/zahradnik/PycharmProjects/master_thesis')

In [2]:
import numpy as np
import pandas as pd
import os
from src.modules import conf, fit, df_columns, evl, preprocess
from sklearn.model_selection import train_test_split
from datetime import datetime
from xgboost import XGBRegressor

/home/zahradnik/PycharmProjects/master_thesis/src/modules/__init__.py
/home/zahradnik/PycharmProjects/master_thesis


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
race_day = '2023-12-03-11-30'

In [5]:
data = fit.load_pcls(
        'zahradnik',
        'running',
        conf["Paths"]["pcl"],
        datetime.strptime(race_day, "%Y-%m-%d-%H-%M")
    )
test_df = data[31]
data.pop(31)
print("")

100%|██████████| 386/386 [00:00<00:00, 493.66it/s]

In [6]:
cad_test = test_df["cadence"]
hr_test = test_df["heart_rate"]
speed_test = test_df["enhanced_speed"]
cols_to_drop = ["cadence","heart_rate","enhanced_speed"]
for x in cols_to_drop:
    cols_drop = test_df.filter(like=x).columns
    test_df.drop(columns=cols_drop,inplace=True)

In [7]:
low_dist, high_dist = preprocess.segment_data(data)
len(low_dist),len(high_dist)

(144, 241)

In [8]:
if max(test_df.distance) > 10000:
    train_df = fit.clean_data(pd.concat(high_dist))
else:
    train_df = fit.clean_data(pd.concat(low_dist))
len(train_df)

681291

In [21]:
# Objective function for XGBoost regression
def objective_function(params):
    # Extract hyperparameters
    learning_rate, max_depth, min_child_weight = params
    
    # Train XGBoost model
    model = xgb.XGBRegressor(learning_rate=abs(learning_rate), max_depth=round(max_depth), min_child_weight=min_child_weight, random_state=42)
    model.fit(X_train, y_train)

    # Predict on the validation set
    y_pred = model.predict(X_valid)

    # Calculate mean squared error (you can replace this with your own evaluation metric)
    mae = mean_absolute_error(y_valid, y_pred)

    # Return mae since we are maximizing the objective
    return mae

# Random shooting

In [13]:
import numpy as np
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
from tqdm import tqdm

def random_search(iterations, search_range):
    # Initialize best parameters and score
    best_params = None
    best_score = float('-inf')

    for iteration in tqdm(range(iterations)):
        # Sample random hyperparameters in the specified search range
        current_params = [np.random.uniform(low, high) for low, high in search_range]

        # Evaluate the objective function at the current point
        current_score = objective_function(current_params)

        # Update the best point if the current point is better
        if current_score > best_score:
            best_params = current_params
            best_score = current_score

    return best_params, best_score
# Example usage
iterations = 2
search_range = [(0.001, 0.2), (3, 10), (1, 5)]

In [26]:
# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_df[test_df.columns], test_df, train_df["cadence"], cad_test 

best_params, best_score = random_search(iterations, search_range)

print("Best Parameters:", best_params)
print("Best Score for Cadence (Mean Absolute Error):", best_score)

test_df["cadence"] = cad_test
test_df= preprocess.calc_windows(df=test_df,
          lagged=15,
          cols=["cadence"])
test_df = preprocess.calc_moving(df=test_df, max_range=110, col="cadence")

100%|██████████| 2/2 [00:38<00:00, 19.47s/it]

Best Parameters: [0.08677608898855552, 8.36217724971346, 4.669438530333382]
Best Score for Cadence (Mean Absolute Error): 0.9476478259575504


In [27]:
# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_df[test_df.columns], test_df, train_df["heart_rate"], hr_test 

best_params, best_score = random_search(iterations, search_range)

print("Best Parameters:", best_params)
print("Best Score for Cadence (Mean Absolute Error):", best_score)


from feature_engine.creation import MathFeatures
test_df["heart_rate"] = hr_test
for fce in ["sum", "mean", "min", "max"]:
    test_df = MathFeatures(variables=["heart_rate", "cadence"], func=fce).fit(test_df).transform(test_df)
    
test_df= preprocess.calc_windows(df=test_df,
          lagged=12,
          cols=["heart_rate"])
test_df = preprocess.calc_moving(df=test_df, max_range=110, col="heart_rate")

100%|██████████| 2/2 [00:41<00:00, 20.52s/it]

Best Parameters: [0.1306719659914615, 7.174196967658485, 1.9274628940600311]
Best Score for Heart rate (Mean Absolute Error): 5.387001972396233


In [28]:
# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_df[test_df.columns], test_df, train_df["enhanced_speed"], speed_test 

best_params, best_score = random_search(iterations, search_range)

print("Best Parameters:", best_params)
print("Best Score for Speed (Mean Absolute Error):", best_score)

100%|██████████| 2/2 [00:36<00:00, 18.06s/it]

Best Parameters: [0.19829379266243014, 9.750791581598133, 1.4681710785859101]
Best Score for Speed (Mean Absolute Error): 0.9730259437348425


## Genetics optimization with DEAP

In [66]:
from deap import base, creator, tools, algorithms
toolbox = base.Toolbox()
import random
attr_set = []
search_space = [(0.001, 0.2), (3, 10), (1, 5)]

for key,counter in zip(search_space,range(len(search_space))):
    print(key)
    if isinstance(key[0],float):
        toolbox.register(f"attr_{counter}", random.uniform, key[0], key[1])
    elif isinstance(key[0],int):
        toolbox.register(f"attr_{counter}", random.randint, key[0], key[1])

    attr_set.append(getattr(toolbox, (f"attr_{counter}")))
attr_set

(0.001, 0.2)
(3, 10)
(1, 5)


[functools.partial(<bound method Random.uniform of <random.Random object at 0x5636caf04b10>>, 0.001, 0.2),
 functools.partial(<bound method Random.randint of <random.Random object at 0x5636caf04b10>>, 3, 10),
 functools.partial(<bound method Random.randint of <random.Random object at 0x5636caf04b10>>, 1, 5)]

In [94]:
import numpy as np
import xgboost as xgb
from deap import base, creator, tools, algorithms

def genetic_opt(pop_size: int, genetic_iterations: int, search_space: list):
    # Define individual and population
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    
    toolbox = base.Toolbox()
    toolbox.register("attr_float", np.random.uniform, abs(search_space[0][0]), abs(search_space[0][1]))
    toolbox.register("attr_int1", np.random.randint, search_space[1][0], search_space[1][1])
    toolbox.register("attr_int2", np.random.randint, search_space[2][0], search_space[2][1])
    toolbox.register("individual", tools.initCycle, creator.Individual, attr_set, n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # Register genetic operators
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    # Evaluation function
    toolbox.register("evaluate", objective_function)
    
    # Main genetic algorithm loop
    population = toolbox.population(n=pop_size)
    
    for gen in tqdm(range(genetic_iterations)):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.2)
        fits = toolbox.map(toolbox.evaluate, offspring)
        
        for ind, fit in zip(offspring, fits):
            ind.fitness.values = [fit]
    
        population = toolbox.select(offspring, k=len(population))
    best_individual = tools.selBest(population, k=1)[0]
    return tools.selBest(population, k=1)[0],  best_individual[0]
    
# Genetic algorithm parameters
pop_size = 5
genetic_iterations = 2
search_space = [(0.001, 0.2), (3, 10), (1, 5)]  # Search range for learning_rate, max_depth, and min_child_weight 

In [95]:
X_train, X_valid, y_train, y_valid = train_df[test_df.columns], test_df, train_df["cadence"], cad_test 

# Get the best individual from the final population
best_individual, best_params = genetic_opt(pop_size, genetic_iterations, search_space)

print("Best Parameters:", best_params)
print("Best Score (Mean Absolute Error):", -best_individual.fitness.values[0])

test_df["cadence"] = cad_test
test_df= preprocess.calc_windows(df=test_df,
          lagged=15,
          cols=["cadence"])
test_df = preprocess.calc_moving(df=test_df, max_range=110, col="cadence")

  0%|          | 0/2 [00:00<?, ?it/s]

Training: [0.0003635077937490464, 8.09030446018643, 0.7402927430822448]
Training: [0.02159084217795138, 7.603431790105426, 6.495445670820286]
Training: [0.03235802349277395, 3, 4]
Training: [0.07197449905347374, 6, 1]
Training: [0.6571753049738144, 10, 4]


 50%|█████     | 1/2 [05:42<05:42, 342.57s/it]

Training: [0.6571753049738144, 10.0, 4.0]
Training: [0.6571753049738144, 10.0, 4.0]
Training: [-0.26828264389743367, 10.69886067623877, 4.0]
Training: [0.9578159723640219, 2.3011393237612316, 4.0]
Training: [0.6571753049738144, 10, 4]


100%|██████████| 2/2 [12:00<00:00, 360.18s/it]

Best Parameters: -0.26828264389743367
Best Score (Mean Absolute Error): -1.2923004818439856


In [96]:
X_train, X_valid, y_train, y_valid = train_df[test_df.columns], test_df, train_df["heart_rate"], hr_test 
best_params, best_score = genetic_opt(pop_size, genetic_iterations, search_space)
print("Best Parameters for heart_rate:", best_params)
print("Best Score for heart_rate (Mean Absolute Error):",best_score)

from feature_engine.creation import MathFeatures
test_df["heart_rate"] = hr_test
for fce in ["sum", "mean", "min", "max"]:
    test_df = MathFeatures(variables=["heart_rate", "cadence"], func=fce).fit(test_df).transform(test_df)
    
test_df= preprocess.calc_windows(df=test_df,
          lagged=12,
          cols=["heart_rate"])
test_df = preprocess.calc_moving(df=test_df, max_range=110, col="heart_rate")

  0%|          | 0/2 [00:00<?, ?it/s]

Training: [0.0667557457343446, 8.55171987612435, 1.8754143892186315]
Training: [0.04955287691688726, 1.4482801238756493, 3.1245856107813683]
Training: [0.04970787744152876, 2.946086916373445, 4.0]
Training: [0.20908222595742815, 10.053913083626554, 4.0]
Training: [0.06107347120121371, 8, 3]


 50%|█████     | 1/2 [05:13<05:13, 313.48s/it]

Training: [0.18482394252982265, 4.041646443176392, 4.0]
Training: [0.07396616086913428, 8.958353556823607, 4.0]
Training: [-0.10119428461194085, 6.2721716393496125, 5.273811871754961]
Training: [-0.010088494145591959, 5.2300215681525914, 3.6777477980110627]
Training: [0.04955287691688726, 1.4482801238756493, 3.1245856107813683]


100%|██████████| 2/2 [09:10<00:00, 275.41s/it]

Best Parameters for heart_rate: [-0.010088494145591959, 5.2300215681525914, 3.6777477980110627]
Best Score for heart_rate (Mean Absolute Error): -0.010088494145591959


In [97]:
X_train, X_valid, y_train, y_valid = train_df[test_df.columns], test_df, train_df["enhanced_speed"], speed_test 
best_params, best_score = genetic_opt(pop_size, genetic_iterations, search_space)
print("Best Parameters for enhanced_speed:", best_params)
print("Best Score for enhanced_speed (Mean Absolute Error):",best_score)

  0%|          | 0/2 [00:00<?, ?it/s]

Training: [0.06957408921686217, 3.7702798144844314, 1.1441528080553276]
Training: [0.04414249853719565, 3.2297201855155686, 5.855847191944672]
Training: [0.11225700408119613, 7.0, 3.9480862644362573]
Training: [0.11475660073441396, 7.0, 1.0519137355637427]
Training: [-0.5425037351938867, 8, 3]


 50%|█████     | 1/2 [04:45<04:45, 285.11s/it]

Training: [-0.4575236238066777, 8.341211587748305, 2.528154272332778]
Training: [0.027276892693987115, 6.658788412251695, 4.419931992103479]
Training: [-0.44124013075880253, 8.373002833814741, 4.093921391473539]
Training: [0.010993399646111901, 6.626997166185259, 2.854164872962719]
Training: [-0.5425037351938867, 8, 3]


100%|██████████| 2/2 [10:52<00:00, 326.30s/it]

Best Parameters for enhanced_speed: [0.010993399646111901, 6.626997166185259, 2.854164872962719]
Best Score for enhanced_speed (Mean Absolute Error): 0.010993399646111901


# Simulated Annealing

In [43]:
import math
import numpy as np
import xgboost as xgb
import random
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
def simulated_annealing(initial_params, objective_function, temperature, cooling_rate, num_iterations):
    current_params = np.array(initial_params)
    current_cost = objective_function(current_params)

    best_params = current_params
    best_cost = current_cost
    all_params = []
    obj_values= []
    all_params.append(current_params)
    for iteration in tqdm(range(num_iterations)):
        # Generate a neighboring solution
        new_params = current_params + np.random.uniform(-0.1, 0.1, size=len(current_params))

        # Evaluate the objective function for the new solution
        new_cost = objective_function(new_params)

        # Accept the new solution if it's better or with a certain probability if it's worse
        if new_cost < current_cost or random.uniform(0, 1) < math.exp((current_cost - new_cost) / temperature):
            current_params = new_params
            current_cost = new_cost

        # Update the best solution if needed
        if current_cost < best_cost:
            best_params = current_params
            best_cost = current_cost

        # Reduce the temperature
        temperature *= cooling_rate
        obj_values.append(current_cost)
        all_params.append(current_params)

    return best_params, best_cost, all_params, obj_values

In [45]:
import matplotlib.pyplot as plt

def visualize_optimization(iterations, parameter_values, objective_values):
    # Plotting the values of each parameter over iterations
    plt.figure(figsize=(10, 6))
    for param_idx in range(len(parameter_values[0])):
        plt.plot(range(iterations+1), [params[param_idx] for params in parameter_values], label=f'Parameter {param_idx + 1}')

    plt.title('Optimization Process - Parameter Values')
    plt.xlabel('Iteration')
    plt.ylabel('Parameter Value')
    plt.legend()
    plt.show()

    # Plotting the objective function values over iterations
    plt.figure(figsize=(10, 6))
    plt.plot(range(iterations), objective_values, label='Objective Function Value', color='red')

    plt.title('Optimization Process - Objective Function')
    plt.xlabel('Iteration')
    plt.ylabel('Objective Function Value')
    plt.legend()
    plt.show()

In [ ]:

# Example usage
X_train, X_valid, y_train, y_valid = train_df[test_df.columns], test_df, train_df["cadence"], cad_test 

initial_params = [0.1, 5, 3]  # Initial values for learning_rate, max_depth, min_child_weight
temperature = 1.0
cooling_rate = 0.95
num_iterations = 100

best_params, best_cost,all_params,obj_values = simulated_annealing(initial_params, objective_function, temperature, cooling_rate, num_iterations)

print("Best Parameters:", best_params)
print("Best Score (Mean Absolute Error):", best_cost)
test_df["cadence"]= cad_test
test_df= preprocess.calc_windows(df=test_df,
          lagged=15,
          cols=["cadence"])
test_df = preprocess.calc_moving(df=test_df, max_range=110, col="cadence")
# Example usage:
# Assuming you have recorded parameter_values and objective_values during optimization
visualize_optimization(num_iterations, all_params[:10], obj_values[:10])

 48%|████▊     | 48/100 [11:45<11:36, 13.40s/it]

In [ ]:
# Example usage
X_train, X_valid, y_train, y_valid = train_df[test_df.columns], test_df, train_df["heart_rate"], cad_test 

initial_params = [0.1, 5, 3]  # Initial values for learning_rate, max_depth, min_child_weight
temperature = 1.0
cooling_rate = 0.95
num_iterations = 100

best_params, best_cost = simulated_annealing(initial_params, objective_function, temperature, cooling_rate, num_iterations)

print("Best Parameters:", best_params)
print("Best Score (Mean Absolute Error):", best_cost)

test_df["heart_rate"]=hr_test
from feature_engine.creation import MathFeatures
test_df["heart_rate"] = hr_test
for fce in ["sum", "mean", "min", "max"]:
    test_df = MathFeatures(variables=["heart_rate", "cadence"], func=fce).fit(test_df).transform(test_df)
    
test_df= preprocess.calc_windows(df=test_df,
          lagged=12,
          cols=["heart_rate"])
test_df = preprocess.calc_moving(df=test_df, max_range=110, col="heart_rate")
# Example usage:
# Assuming you have recorded parameter_values and objective_values during optimization
visualize_optimization(num_iterations, all_params, obj_values)

In [ ]:
# Example usage
X_train, X_valid, y_train, y_valid = train_df[test_df.columns], test_df, train_df["enhanced_speed"], cad_test 

initial_params = [0.1, 5, 3]  # Initial values for learning_rate, max_depth, min_child_weight
temperature = 1.0
cooling_rate = 0.95
num_iterations = 100

best_params, best_cost, all_params, obj_values = simulated_annealing(initial_params, objective_function, temperature, cooling_rate, num_iterations)

print("Best Parameters:", best_params)
print("Best Score (Mean Absolute Error):", best_cost)
# Example usage:
# Assuming you have recorded parameter_values and objective_values during optimization
visualize_optimization(num_iterations, all_params[:10], obj_values[:10])